In [2]:
!pip install ultralytics opencv-python-headless numpy gradio


In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
import gradio as gr


In [4]:
model = YOLO("yolov8n.pt")  # YOLO Nano for speed


In [5]:
def get_car_colour(image, box):
    x1, y1, x2, y2 = map(int, box)
    car_img = image[y1:y2, x1:x2]
    hsv = cv2.cvtColor(car_img, cv2.COLOR_BGR2HSV)

    avg_color = np.mean(hsv.reshape(-1, 3), axis=0)
    hue = avg_color[0]

    if 100 < hue < 130:  # Blue hue range
        return "blue"
    else:
        return "other"


In [6]:
def detect_objects(image):
    results = model(image)
    annotated = image.copy()
    car_count, person_count = 0, 0

    for r in results[0].boxes:
        cls = int(r.cls)
        label = model.names[cls]
        x1, y1, x2, y2 = r.xyxy[0]

        if label == "car":
            car_count += 1
            colour = get_car_colour(image, (x1, y1, x2, y2))
            rect_color = (0, 0, 255) if colour == "blue" else (255, 0, 0)
            cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), rect_color, 2)

        elif label == "person":
            person_count += 1
            cv2.rectangle(annotated, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    cv2.putText(annotated, f"Cars: {car_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(annotated, f"People: {person_count}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    return annotated


In [ ]:
def process_image(input_image):
    img = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
    annotated = detect_objects(img)
    return cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

gr.Interface(fn=process_image,
             inputs=gr.Image(type="numpy"),
             outputs="image",
             title="Car Colour & People Detection").launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a0aeff425d623709a1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 480x640 1 car, 440.0ms
Speed: 40.7ms preprocess, 440.0ms inference, 38.4ms postprocess per image at shape (1, 3, 480, 640)
